In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree

In [3]:
# bank=pd.read_excel("/content/drive/MyDrive/Imarticus/PGA44/Decision_tree/bank.xlsx", sheet_name="bank")
bank=pd.read_csv("/content/drive/MyDrive/Imarticus/PGA46/ML/dt/NewCustomer.csv")

In [11]:
a=bank.columns.to_list()
a1=list(map(lambda x: x.lower(), a))
bank.columns=a1
bank.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,deposit
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no


In [27]:
bank.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'empvarrate', 'conspriceidx', 'consconfidx', 'euribor3m',
       'nremployed', 'deposit'],
      dtype='object')

In [12]:
bank["job"]=bank["job"].replace(["unemployed", "student", "retired"], "Not_earning")
bank["job"]=bank["job"].replace(["housemaid", "services", "admin."], "pink_collar")
bank["job"]=bank["job"].replace(["self-employed", "entrepreneur"], "Self_earning")

In [13]:
bank.drop(columns=["month"], inplace=True)

In [14]:
bank["deposit"]=bank["deposit"].map({"yes":1, "no":0})

In [15]:
# Outlier capping
def Outliers(x):
  x=x.clip(upper=x.quantile(.99))
  x=x.clip(lower=x.quantile(.01))
  return x

In [16]:
num=bank.select_dtypes("int").apply(Outliers)  # outlier treatment done

In [17]:
# num
ob=bank.select_dtypes("object")

In [18]:
bank_data=pd.concat([num, ob], axis=1)

In [19]:
final_data=pd.get_dummies(bank_data,
                          columns=['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact','poutcome'],
                          drop_first=True)

In [20]:
final_data.shape

(319, 28)

In [21]:
import os
os.chdir("/content/drive/MyDrive/Imarticus/PGA44/Decision_tree")

In [23]:
# Saving the model for future use
from joblib import dump, load
dt_model=load("dt4.joblib")
col_used=load("selected_feat.joblib")

In [24]:
col_used

array(['duration', 'poutcome_success', 'contact_unknown', 'housing_yes',
       'balance', 'campaign', 'age', 'pdays', 'day', 'loan_yes'],
      dtype=object)

In [28]:
final_data["poutcome_success"]=0
final_data["contact_unknown"]=0
final_data["balance"]=0
final_data["day"]=0

In [31]:
x=final_data[col_used]
pred_result=dt_model.predict(x)

In [40]:
prob_results=pd.DataFrame(dt_model.predict_proba(x), columns=["prob_0", "prob_1"])
# prob_results.head()

In [36]:
result_data=bank.copy()

In [37]:
result_data["pred_result"]=pred_result
result_data["prob_for_no_fd"]=prob_results["prob_0"]
result_data["prob_for_fd"]=prob_results["prob_1"]

In [38]:
result_data.head()

,age,job,marital,education,default,housing,loan,contact,day_of_week,duration,...,poutcome,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,deposit,pred_result,prob_for_no_fd,prob_for_fd
0,56,pink_collar,married,basic.4y,no,no,no,telephone,mon,261,...,nonexistent,1.1,93.994,-36.4,4.857,5191,0,0,0.620690,0.379310
1,57,pink_collar,married,high.school,unknown,no,no,telephone,mon,149,...,nonexistent,1.1,93.994,-36.4,4.857,5191,0,0,0.774194,0.225806
2,37,pink_collar,married,high.school,no,yes,no,telephone,mon,226,...,nonexistent,1.1,93.994,-36.4,4.857,5191,0,1,0.041667,0.958333
3,40,pink_collar,married,basic.6y,no,no,no,telephone,mon,151,...,nonexistent,1.1,93.994,-36.4,4.857,5191,0,0,0.774194,0.225806
4,56,pink_collar,married,high.school,no,no,yes,telephone,mon,307,...,nonexistent,1.1,93.994,-36.4,4.857,5191,0,0,0.620690,0.379310


In [42]:
result_data[result_data["pred_result"]==1].shape

(95, 23)

In [43]:
result_data.shape

(319, 23)